In [16]:
%load_ext autoreload
%autoreload 2

import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense, Flatten, Input, Reshape
from keras.models import Sequential, Model
from PIL import Image
from pprint import pprint

from keras_preprocessing.image import DataFrameIterator, ImageDataGenerator


pixel_val = 1
np_filenames, img_filenames, np_arrays = [], [], []
for i in range(5):
    np_filename = '/tmp/{}.npy'.format(i)
    img_filename = '/tmp/{}.jpg'.format(i)
    array = pixel_val * np.random.randint(0, 255, size=(2, 2)).astype('float32')
    np.save(np_filename, array[..., None])
    plt.imsave(img_filename, array)
    
    np_filenames.append(np_filename)
    img_filenames.append(img_filename)
    
df = pd.DataFrame({'img_path': img_filenames, 'img_np_path': np_filenames})
df['regression'] = np.random.randint(1, 10, size=len(df))
df['binary'] = random.sample(['dog', 'cat'] * 30, len(df))
df['multi_class'] = random.sample(['dog', 'cat', 'horse'] * 30, len(df))
df['multi_label'] = random.sample(['dog', 'cat', ['dog'], ['cat'], ['cat', 'dog'], ['dog', 'cat']] * 10, len(df))
df['img_path_extra'] = img_filenames
df['regression_extra'] = np.random.randint(10, 100, size=len(df))

df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,img_path,img_np_path,regression,binary,multi_class,multi_label,img_path_extra,regression_extra
0,/tmp/0.jpg,/tmp/0.npy,2,dog,dog,[dog],/tmp/0.jpg,14
1,/tmp/1.jpg,/tmp/1.npy,9,cat,horse,dog,/tmp/1.jpg,15
2,/tmp/2.jpg,/tmp/2.npy,3,dog,dog,[cat],/tmp/2.jpg,15
3,/tmp/3.jpg,/tmp/3.npy,8,cat,cat,dog,/tmp/3.jpg,85
4,/tmp/4.jpg,/tmp/4.npy,5,dog,cat,[dog],/tmp/4.jpg,58


String columns as input

In [13]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns='regression',
    image_size=(2, 2),
    batch_size=2
)
print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(2, 2, 3)))
model.add(Dense(1))
model.compile('adam', 'mse')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg']}]

Outputs:
[{'column': 'regression', 'mode': None, 'values': [2, 2, 4, 1, 5]}]
Epoch 1/1
3/3 [==============================] - 0s 29ms/step - loss: 54695.7107


Numpy array image and column name in a list

In [14]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_np_path',
    output_columns=['regression'],
    image_size=(2, 2),
    batch_size=2
)
print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(2, 2, 3)))
model.add(Dense(1))
model.compile('adam', 'mse')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_np_path',
  'values': ['/tmp/0.npy',
             '/tmp/1.npy',
             '/tmp/2.npy',
             '/tmp/3.npy',
             '/tmp/4.npy']}]

Outputs:
[{'column': 'regression', 'mode': None, 'values': [2, 2, 4, 1, 5]}]
Epoch 1/1
3/3 [==============================] - 0s 32ms/step - loss: 21638.3620


with data augmentation

In [12]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_np_path',
    output_columns=['regression'],
    image_size=(2, 2),
    batch_size=2,
    image_data_generator=ImageDataGenerator(horizontal_flip=True,
                                            vertical_flip=True)
)
print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(2, 2, 3)))
model.add(Dense(1))
model.compile('adam', 'mse')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_np_path',
  'values': ['/tmp/0.npy',
             '/tmp/1.npy',
             '/tmp/2.npy',
             '/tmp/3.npy',
             '/tmp/4.npy']}]

Outputs:
[{'column': 'regression', 'mode': None, 'values': [2, 2, 4, 1, 5]}]
Epoch 1/1
3/3 [==============================] - 0s 28ms/step - loss: 16026.2403
